<a href="https://colab.research.google.com/github/UCI-CHRS/GEE-Training-2025/blob/main/notebooks/Tutorial01_Access_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

## **Remote Sensing Datasets: Access and Download for Area of Interest**  
### **PEER2PEER GEE Training**  
**Developed by:** Muhammad Umar Akbar<sup>1</sup>, Annika Hjelmstad<sup>2</sup>, Debora Yumi de Oliveira<sup>2,3</sup>, Kasra Khodkar<sup>1</sup>  
**Affiliation:** <sup>1</sup>Department of Biosystems and Agricultural Engineering, Oklahoma State University, OK, USA  
<sup>2</sup>Department of Civil and Environmental Engineering, University of California at Irvine, Irvine, California, USA
<sup>3</sup>Mackenzie Presbyterian University, São Paulo, Brazil

</center>

---

## **Introduction**

Welcome to this Google Earth Engine (GEE) training notebook! In this guide, we will walk you through the process of accessing, visualizing, and downloading various remote sensing datasets available in the Google Earth Engine Data Catalog.

Remote sensing provides valuable insights into environmental and agricultural monitoring, hydrology, climate studies, and land management. This notebook will cover key datasets, including:

1. Elevation (Digital Elevation Models, DEMs)  
2. Land Use Land Cover (LULC)  
3. Precipitation
4. Temperature (Land Surface Temperature)  
5. Soil Moisture
---

First, we need to import the Python modules we will use:

In [ ]:
# Import necessary Python modules
import ee       # ee provides functions for sending requests to the GEE servers
import geemap   # geemap is for mapping ee objects

Next, we need to authenticate and initialize with the Google Cloud project we created previously.

In [ ]:
# Authenticate and initialize with your google cloud project
ee.Authenticate()
ee.Initialize(project='annikas-tutorial') # replace the project ID with yours

<table>
  <tr>
    <td><img src="https://drive.google.com/uc?id=1ohsAbSz21MdnZm0t688q4QYCeHt71H5B" width="300"></td>
    <td><img src="https://drive.google.com/uc?id=1ysOPRUQd-2x1uPa6WZZu3Ymxsm8eozag" width="300"></td>
  </tr>
  <tr>
    <td><img src="https://drive.google.com/uc?id=1k-LHDBaFx2NMKVczqjEfRv1-9pvphncF" width="300"></td>
    <td><img src="https://drive.google.com/uc?id=1GfFSv5hJDeW1dxMWqTLkoEN8tdQf3AI_" width="300"></td>
  </tr>
</table>


Now we're ready to read in the data. Let's define our area of interest (AOI).

In [ ]:
# Get Tanzania country boundary
tanzania = ee.FeatureCollection("UN/Geodata/BNDA_simplified/current").filter('iso2cd == "TZ"')

# Define a rectangle AOI in Tanzania
aoi = ee.Geometry.Rectangle([33.0, -4.5, 34.5, -3.0])  # Adjust coordinates as needed

# Create interactive map and display AOI
Map = geemap.Map()
Map.centerObject(tanzania,6)
Map.addLayer(tanzania, {'color': '808080'}, 'Tanzania')
Map.addLayer(aoi, {'color': 'ff0000'}, 'AOI')
Map

## **1. Elevation**

For this tutorial, we will explore the [NASA SRTM Digital Elevation Model (30m)](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003) dataset.

The Shuttle Radar Topography Mission (SRTM) digital elevation data is an international research effort that obtained digital elevation models on a near-global scale. This SRTM V3 product (SRTM Plus) is provided by NASA JPL at a resolution of 1 arc-second (approximately 30 m).

This dataset has undergone a void-filling process using open-source data sources such as ASTER GDEM2, GMTED2010, and NED, as opposed to other versions that contain voids or have been filled with commercial sources.

Earth Engine Snippet: `ee.Image("USGS/SRTMGL1_003")`

<img src="https://drive.google.com/uc?id=1fpNNh78IcIVp_jk4Ptk2HHRxJG4dnMWh" width="500">


In [ ]:
# Load the SRTM 30m DEM
dem = ee.Image("USGS/SRTMGL1_003")

# Clip the dataset to AOI
dem_aoi = dem.clip(aoi)

In [ ]:
# Visualization parameters
vis_params = {
    'min': 1000,
    'max': 1600,
    'palette': ['white', 'green', 'brown']
}

# Create interactive map and display DEM
Map1 = geemap.Map()
Map1.centerObject(aoi, 9)
Map1.addLayer(dem_aoi, vis_params, 'SRTM Elevation')

# Add legend to the map (categorical)
legend_dict = {
    "Low (1000m)": "ffffff",
    "Medium (~1300m)": "008000",
    "High (1600m+)": "8b4513"
}

# Generate and attach legend
Map1.add_legend(
    title="SRTM Elevation",
    legend_dict=legend_dict,
    draggable=False,
    output="srtm_legend.html",
)

# Display the map
Map1

## **2. Land Use Land Cover (LULC)**

Land Use Land Cover (LULC) refers to the physical characteristics of the Earth's surface (land cover) and how humans utilize that surface (land use). For this tutorial, we will explore the [ESA WorldCover 10m Land Cover Map](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200) dataset.

The European Space Agency (ESA) WorldCover 10 m 2021 product provides a global land cover map at a high spatial resolution of 10 meters, derived from Sentinel-1 and Sentinel-2 satellite data. This dataset includes 11 distinct land cover classes and was developed as part of the [ESA WorldCover](https://esa-worldcover.org/en) project, under the 5th Earth Observation Envelope Programme (EOEP-5).

The WorldCover 2021 v200 version delivers improved classification and thematic detail, achieving an overall accuracy of 76.7%. This product is especially valuable for applications in environmental monitoring, land use planning, and ecosystem assessment on both global and regional scales.

Earth Engine Snippet: `ee.ImageCollection("ESA/WorldCover/v200")`

<img src="https://drive.google.com/uc?id=12eDvp9crju4oVCXMH9ODMTrbTV3l4UFp" width="500">

In [ ]:
# Load the ESA WorldCover v200 dataset and get the first image
land_cover = ee.ImageCollection('ESA/WorldCover/v200').first()

# Clip the dataset to AOI
land_cover_aoi = land_cover.clip(aoi)

In [ ]:
# Define visualization parameters
vis_params = {'bands': ['Map']}

# Create and display the map
Map2 = geemap.Map()
Map2.centerObject(aoi, 9)
Map2.addLayer(land_cover_aoi, vis_params, 'ESA WorldCover v200')

# Add the built-in legend to the map
Map2.add_legend(
    title="ESA Land Cover Type",
    builtin_legend="ESA_WorldCover",
    draggable=False,
    position="bottomleft",
    style={"bottom": "5px"},
)

# Display the map
Map2

## **3. Precipitation**

For this tutorial, we will explore two precipitation datasets:

* [PERSIANN-CDR](https://developers.google.com/earth-engine/datasets/catalog/NOAA_PERSIANN-CDR)
* [CHIRPS: Rainfall Estimates from Rain Gauge and Satellite Observations](https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY)



### **PERSIANN-CDR**

PERSIANN-CDR is a daily quasi-global precipitation product that spans the period from 1983-01-01 to present. The data is produced quarterly, with a typical lag of three months. The product is developed by the Center for Hydrometeorology and Remote Sensing at the University of California, Irvine ([UC-IRVINE/CHRS](https://chrs.web.uci.edu/)).

Earth Engine Snippet: `ee.ImageCollection("NOAA/PERSIANN-CDR")`

<img src="https://drive.google.com/uc?export=download&id=1wN4xDYPuNx-_9uW8WJMUgH9Dd9COykTF" width="700">

### **CHIRPS: Rainfall Estimates from Rain Gauge and Satellite Observations**

The Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS) is a long-term, high-resolution rainfall dataset developed to support drought early warning and climate monitoring efforts. Covering the region from 50°S to 50°N globally and extending from 1981 to the near-present, CHIRPS blends multiple data sources including satellite-based infrared imagery, in-situ rain gauge observations, and a high-resolution climatology known as CHPclim.

With a spatial resolution of 0.05° (~5 km), CHIRPS provides gridded rainfall time series ideal for trend analysis, drought assessment, and seasonal forecasting. The dataset was developed in collaboration with the USGS Earth Resources Observation and Science (EROS) Center, aiming to provide consistent, reliable, and near-real-time precipitation data for regions with limited ground-based observations.

Earth Engine Snippet: `ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")`


<img src="https://drive.google.com/uc?id=1xQI77ShzdPIRk6AdigkNXjT3QdjdoExn" width="700">

In [ ]:
# Load PERSIANN-CDR daily data and filter for the year 2018
persiann = ee.ImageCollection('NOAA/PERSIANN-CDR') \
    .filterDate('2018-01-01', '2018-12-31')

# Load CHIRPS daily data and filter for the year 2018
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate('2018-01-01', '2018-12-31')

Let's see what's in the image collection. (Click on the dropdown arrows after running the following cell.)

In [ ]:
persiann # this will print out information about the variable

In [ ]:
chirps # this will print out information about the variable

In [ ]:
# Select the precipitation band, sum over the year, and clip to AOI
yearly_precip_persiann = persiann.select('precipitation').sum().clip(aoi)
yearly_precip_chirps = chirps.select('precipitation').sum().clip(aoi)

In [ ]:
# Visualization parameters
vis_params = {
    'min': 600,
    'max': 1200,
    'palette': 'Blues',
}

# Create map and add layer
Map3 = geemap.Map()
Map3.centerObject(aoi, 9)
Map3.addLayer(yearly_precip_persiann, vis_params, 'PERSIANN Yearly Rainfall (2018)')
Map3.addLayer(yearly_precip_chirps, vis_params, 'CHIRPS Yearly Rainfall (2018)')

# Add a vertical colorbar legend to the map
Map3.add_colorbar(
    vis_params,
    label="Precipitation (mm)",
    layer_name="Yearly Rainfall",
    orientation="vertical",
    position="bottomleft",
    transparent_bg=True,
)

# Display the map
Map3

## **4. Temperature**

For this tutorial, we will explore the [MODIS Land Surface Temperature and Emissivity (MOD11A1 V6.1)](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1) dataset.

The MOD11A1 V6.1 product provides daily land surface temperature (LST) and emissivity data at a spatial resolution of 1 km, organized into a 1200 × 1200 km global grid. The LST values are derived from the MOD11_L2 swath product, which utilizes thermal infrared measurements from the MODIS instrument onboard NASA’s Terra satellite.

In regions above 30° latitude, pixels may include multiple clear-sky observations within a day. When this occurs, the dataset returns the average value of all qualifying measurements for that pixel. The product includes both daytime and nighttime LST bands, their associated quality assurance layers, as well as MODIS thermal bands 31 and 32 and six observation layers that provide additional context on viewing geometry and cloud cover conditions.

This dataset is useful for a wide range of applications including climate monitoring, evapotranspiration modeling, and land-atmosphere interaction studies.

Earth Engine Snippet: `ee.ImageCollection("MODIS/061/MOD11A1")`

<img src="https://drive.google.com/uc?id=14q5w1laScUSUrY-DGpNpDSVMVNC6vNFl" width="700">

In [ ]:
# Load MODIS MOD11A1 V6.1 Land Surface Temperature dataset and filter by date
lst = ee.ImageCollection('MODIS/061/MOD11A1') \
    .filterDate('2018-01-01', '2018-05-01')

# Select the daytime LST band, calculate mean, apply scale factor, and clip to AOI
lst_day = lst.select('LST_Day_1km').mean().multiply(0.02).clip(aoi)

In [ ]:
# Visualization parameters
vis_params = {
    'min': 297.0,
    'max': 312.0,
    'palette': [
        '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
        'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ]
}

# Create the interactive map
Map4 = geemap.Map()
Map4.centerObject(aoi, 9)
Map4.addLayer(lst_day, vis_params, 'LST Daytime (MODIS) Jan–May 2018')

# Add a vertical colorbar legend to the map
Map4.add_colorbar(
    vis_params,
    label="Land Surface Temperature (K) ",
    layer_name="LST",
    orientation="vertical",
    position="bottomleft",
    transparent_bg=True,
)

# Display the map
Map4

## **5. Soil Moisture**

For this tutorial, we will explore the [SMAP: Soil Moisture Active Passive Mission](https://developers.google.com/earth-engine/datasets/catalog/NASA_SMAP_SPL3SMP_E_005) dataset.

The Soil Moisture Active Passive (SMAP) mission is a NASA satellite observatory designed to measure the amount of water in the top 5 cm of soil across the globe. Launched in January 2015 and operational since April 2015, SMAP provides vital data for understanding the water, energy, and carbon cycles. Although its radar instrument failed shortly after launch, the radiometer continues to deliver high-quality global soil moisture data.

SMAP collects measurements every 2–3 days, enabling the detection of dynamic soil moisture changes driven by weather events, seasonal variation, and climatic shifts. These observations are crucial for applications such as agriculture, drought monitoring, flood forecasting, and climate modeling. The mission completed its prime 3-year phase in 2018 and continues to operate in an extended mission phase, providing an invaluable dataset for hydrological and environmental research.

Earth Engine Snippet: `ee.ImageCollection("NASA/SMAP/SPL3SMP_E/005")`

<img src="https://drive.google.com/uc?id=1xZ6gSdZAlZEthfVY2N-CsHv1ayDzS5es" width="700">

In [ ]:
# Load SMAP SPL3SMP_E dataset and filter by date
soil_moisture = ee.ImageCollection('NASA/SMAP/SPL3SMP_E/005') \
    .filterDate('2017-04-01', '2017-04-30')

# Select the AM surface soil moisture band, take mean, and clip to AOI
soil_moisture_am = soil_moisture.select('soil_moisture_am').mean().clip(aoi)

In [ ]:
# Visualization parameters
vis_params = {
    'min': 0.1,
    'max': 0.3,
    'palette': ['ff0303',  'efff07', '418504', '0300ff',]
}

# Create and display the map
Map5 = geemap.Map()
Map5.centerObject(aoi, 9)
Map5.addLayer(soil_moisture_am, vis_params, 'SMAP Soil Moisture AM (Apr 2017)')

# Add a vertical colorbar legend to the map
Map5.add_colorbar(
    vis_params,
    label=" Soil Moisture (%) ",
    layer_name="Soil Moisture",
    orientation="vertical",
    position="bottomleft",
    transparent_bg=True,
)

# Display the map
Map5

## **Downloading Datasets to Google Drive**

After processing remote sensing datasets in **Google Earth Engine (GEE)**, you can export the results to **Google Drive** using the `Export.image.toDrive()` function in Python.

This export function allows you to save a processed image (e.g., soil moisture, precipitation, elevation, or temperature) as a **GeoTIFF file**.

Key parameters in the export function include:
  - **`image`**: The Earth Engine image you want to export (e.g., mean soil moisture for April 2017).
  - **`description`**: A name for the export task (used for monitoring).
  - **`folder`**: The name of the folder in your **Google Drive** where the file will be saved.
  - **`fileNamePrefix`**: The name for the output file.
  - **`region`**: The area of interest (**AOI**) to clip the export to.
  - **`scale`**: Spatial resolution of the export (should match the native resolution of the dataset).
  - **`crs`**: Coordinate Reference System (e.g., `'EPSG:4326'` for geographic lat/lon).

- Once these parameters are defined, you can start the export by calling `task.start()`.

- The exported file will appear in your **Google Drive** under the specified folder once processing is complete.

This method is useful for downloading processed datasets for offline analysis, creating maps in GIS software, or sharing with collaborators.



In [ ]:
# Export the image to your Google Drive
task = ee.batch.Export.image.toDrive(
    image=soil_moisture_am,
    description='SMAP_SoilMoisture_Apr2017',
    folder='EarthEngineExports',  # Your Drive folder name
    fileNamePrefix='SMAP_SoilMoisture_Tanzania_Apr2017',
    region=aoi,
    scale=10000,  # SMAP native resolution ~10 km
    crs='EPSG:4326',
    maxPixels=1e13
)

# Start the export task
task.start()

print("Export started. Check Google Drive > EarthEngineExports folder.")

<img src="https://drive.google.com/uc?id=10ZA1_W3UZoz3ST6mMvDyiupUa9q2LjJS" width="600">